In [41]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [42]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [43]:

from pretty_print_sklearn_tree import pretty_print_sklearn_tree

In [44]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

In [45]:
# path = 'data_product_reviews'

### Load training

In [46]:
X_train = pd.read_csv("data_product_reviews/x_train.csv")
y_train = pd.read_csv("data_product_reviews/y_train.csv").values.ravel()

### Load validation set

In [47]:
X_valid = pd.read_csv("data_product_reviews/x_valid.csv")
y_valid = pd.read_csv("data_product_reviews/y_valid.csv").values.ravel()

### Load test set 

In [48]:
X_test  = pd.read_csv("data_product_reviews/x_test.csv")
y_test  = pd.read_csv("data_product_reviews/y_test.csv").values.ravel()

### Load vocabulary as a list of strings

In [49]:
vocab_list = X_train.columns.tolist()
print(f"Vocabulary size: {len(vocab_list)}")
print(f"First 5 words: {vocab_list[:5]}")

Vocabulary size: 7729
First 5 words: ['good', 'great', 'time', 'book', "don't"]


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [50]:
X_combined = np.concatenate([X_train, X_valid])
y_combined = np.concatenate([y_train, y_valid])

# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [51]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

In [52]:
simple_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101
)

### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [53]:
simple_tree.fit(X_train, y_train)

,criterion,'gini'
,splitter,'best'
,max_depth,3
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,101
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [54]:
train_bal_acc = balanced_accuracy_score(y_train, simple_tree.predict(X_train))
valid_bal_acc = balanced_accuracy_score(y_valid, simple_tree.predict(X_valid))
test_bal_acc  = balanced_accuracy_score(y_test,  simple_tree.predict(X_test))

print(f"Balanced Accuracy → Train {train_bal_acc:.3f} | Val {valid_bal_acc:.3f} | Test {test_bal_acc:.3f}")

Balanced Accuracy → Train 0.646 | Val 0.645 | Test 0.646


### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [55]:
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (1 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (1 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (1 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (1 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (1 total training examples)
    N Decision: X['movie'] <= 0.50?
      Y Leaf: p(y

In the above tree no internal node has two leaf children predicting the same sentiment.

But Why Two Children with Same Class Makes Sense
Different Confidence Levels - Even though both leaves predict the same class, they may have different: 
- Class probabilities: Left leaf might be 70% positive, right leaf might be 95% positive 
- Sample sizes: Left leaf has 50 samples, right leaf has 10 samples 
- Gini impurity: Different levels of certainty in the prediction

Having both children predict the same class means the split didn’t change the majority label, but it still improved the confidence or purity of the prediction.

In the bigger tree we can this for eg Both predict NEGATIVE, but with very different confidence levels.

Why This Makes Sense 

Left side: Reviews without "excellent" are strongly negative whereas Right side: Even reviews with "excellent" can be negative (sarcasm, mixed reviews) The split provides valuable information about prediction confidence

## 1B : Find best Decision Tree with grid search

In [56]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit
import numpy as np

In [57]:
# mark train as −1 and validation as 0
test_fold = [-1] * len(X_train) + [0] * len(X_valid)
my_splitter = PredefinedSplit(test_fold=test_fold)

# Define grid of hyperparameters
param_grid = {
    'max_depth': [2, 8, 32, 128],
    'min_samples_leaf': [1, 3, 9],
    'random_state': [101]
}

base_tree = DecisionTreeClassifier(criterion='gini')

grid_search = GridSearchCV(
    estimator=base_tree,
    param_grid=param_grid,
    scoring='balanced_accuracy',
    cv=my_splitter,            
    return_train_score=True,
    refit=False,
    verbose=2
)

grid_search.fit(X_combined, y_combined)

results_df = pd.DataFrame(grid_search.cv_results_)
best_idx   = results_df['mean_test_score'].idxmax()
best_params = results_df.loc[best_idx, 'params']
best_score  = results_df.loc[best_idx, 'mean_test_score']

print("\nBest Decision Tree parameters :", best_params)
print(f"Best Validation Balanced Accuracy : {best_score:.3f}")


Fitting 1 folds for each of 12 candidates, totalling 12 fits
[CV] END ..max_depth=2, min_samples_leaf=1, random_state=101; total time=   2.4s
[CV] END ..max_depth=2, min_samples_leaf=3, random_state=101; total time=   3.0s
[CV] END ..max_depth=2, min_samples_leaf=9, random_state=101; total time=   3.4s
[CV] END ..max_depth=8, min_samples_leaf=1, random_state=101; total time=   7.1s
[CV] END ..max_depth=8, min_samples_leaf=3, random_state=101; total time=   6.8s
[CV] END ..max_depth=8, min_samples_leaf=9, random_state=101; total time=   6.3s
[CV] END .max_depth=32, min_samples_leaf=1, random_state=101; total time=  16.2s
[CV] END .max_depth=32, min_samples_leaf=3, random_state=101; total time=  15.4s
[CV] END .max_depth=32, min_samples_leaf=9, random_state=101; total time=  13.9s
[CV] END max_depth=128, min_samples_leaf=1, random_state=101; total time=  25.0s
[CV] END max_depth=128, min_samples_leaf=3, random_state=101; total time=  22.4s
[CV] END max_depth=128, min_samples_leaf=9, rand

#### The the values of max depth and min samples leaf for the best tree found by the grid search are 32 and 3


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell



In [58]:
# Build the best tree found by grid search and evaluate
best_tree = DecisionTreeClassifier(
    criterion='gini',
    max_depth=best_params['max_depth'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=101
)
best_tree.fit(X_train, y_train)

train_bal_acc = balanced_accuracy_score(y_train, best_tree.predict(X_train))
valid_bal_acc = balanced_accuracy_score(y_valid, best_tree.predict(X_valid))
test_bal_acc  = balanced_accuracy_score(y_test,  best_tree.predict(X_test))

print(f"Best Tree Performance → Train {train_bal_acc:.3f} | Val {valid_bal_acc:.3f} | Test {test_bal_acc:.3f}")



Best Tree Performance → Train 0.877 | Val 0.732 | Test 0.749


### Interpret the best decision tree

In [59]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

The binary tree structure has 817 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    6 are leaves
- depth   5 has   20 nodes, of which    5 are leaves
- depth   6 has   30 nodes, of which   14 are leaves
- depth   7 has   32 nodes, of which   18 are leaves
- depth   8 has   28 nodes, of which   12 are leaves
- depth   9 has   32 nodes, of which   11 are leaves
- depth  10 has   42 nodes, of which   21 are leaves
- depth  11 has   42 nodes, of which   23 are leaves
- depth  12 has   38 nodes, of which   17 are leaves
- depth  13 has   42 nodes, of which   21 are leaves
- depth  14 has   42 nodes, of which   25 are leaves
- depth  15 has   34 nodes, of which   18 are leaves
- depth  16 has   32 nodes, of which   14 are leaves
- depth  17 has   36 nodes, of which   23 are leaves
- dep

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [60]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

### Fit the forest

**TODO Train on the training set** in the next coding cell

In [61]:
simple_forest.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,3
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [62]:
# Evaluate
train_bal_acc = balanced_accuracy_score(y_train, simple_forest.predict(X_train))
valid_bal_acc = balanced_accuracy_score(y_valid, simple_forest.predict(X_valid))
test_bal_acc  = balanced_accuracy_score(y_test,  simple_forest.predict(X_test))

print(f"Balanced Accuracy → Train {train_bal_acc:.3f} | Val {valid_bal_acc:.3f} | Test {test_bal_acc:.3f}")



Balanced Accuracy → Train 0.819 | Val 0.797 | Test 0.778


## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [63]:
import pandas as pd

# Feature importances
importances = simple_forest.feature_importances_
importance_df = pd.DataFrame({
    'word': vocab_list,
    'importance': importances
}).sort_values('importance', ascending=False)

top10 = importance_df.head(10)

# Words with near-zero importance
unimportant_df = importance_df[importance_df['importance'] < 1e-5]
n_unimportant = len(unimportant_df)
random10 = unimportant_df.sample(n=min(10, n_unimportant), random_state=42)

# PD Table
table = pd.DataFrame({
    'Important Words': top10['word'].values,
    'Unimportant Words': random10['word'].values
})
print("Table 2: Feature Importances")
display(table)

print(f"\nTotal features with importance < 1e-5 : {n_unimportant}")


Table 2: Feature Importances


,Important Words,Unimportant Words
0,return,portray
1,excel,bought_one
2,great,entry
3,worst,already_know
4,poor,list
5,disappoint,long_and
6,your_money,exact
7,i_love,and_quite
8,the_best,types_of
9,don't,almost_every



Total features with importance < 1e-5 : 7296


#### Total No of non- zero terms that were eligible were 7296

## 2C: Best Random Forest via grid search



This block might take 2-10 minutes. 

If yours runs significantly longer, try this out on Google Colab instead.

In [64]:
from sklearn.model_selection import GridSearchCV, PredefinedSplit

In [65]:
# Pack training + validation sets into one combined dataset
X_combined = pd.concat([X_train, X_valid])
y_combined = np.concatenate([y_train, y_valid])
test_fold = [-1] * len(X_train) + [0] * len(X_valid)
my_splitter = PredefinedSplit(test_fold=test_fold)


In [66]:
# Define parameter grid
param_grid = {
    'max_features': [3, 10, 33, 100, 333],
    'max_depth': [16, 32],
    'min_samples_leaf': [1],
    'n_estimators': [100],
    'random_state': [101]
}

base_forest = sklearn.ensemble.RandomForestClassifier(
    criterion='gini',
    n_jobs=-1
)

grid_forest = GridSearchCV(
    estimator=base_forest,
    param_grid=param_grid,
    scoring='balanced_accuracy',
    cv=my_splitter,
    return_train_score=True,
    refit=False,
    verbose=2
)

### Do the search!

In [67]:
grid_forest.fit(X_combined, y_combined)

Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.3s
[CV] END max_depth=16, max_features=10, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.2s
[CV] END max_depth=16, max_features=33, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.1s
[CV] END max_depth=16, max_features=100, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.5s
[CV] END max_depth=16, max_features=333, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   3.0s
[CV] END max_depth=32, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.0s
[CV] END max_depth=32, max_features=10, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.0s
[CV] END max_depth=32, max_features=33, min_samples_leaf=1, n_estimators=100, random_state=101; total time=   1.3s
[CV] END max_depth=

,estimator,RandomForestC...ier(n_jobs=-1)
,param_grid,"{'max_depth': [16, 32], 'max_features': [3, 10, ...], 'min_samples_leaf': [1], 'n_estimators': [100], ...}"
,scoring,'balanced_accuracy'
,n_jobs,None
,refit,False
,cv,"PredefinedSpl...hape=(7138,)))"
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,n_estimators,100


### Display search results

In [68]:
results_df = pd.DataFrame(grid_forest.cv_results_)
best_idx = results_df['mean_test_score'].idxmax()
best_params = results_df.loc[best_idx, 'params']
best_score = results_df.loc[best_idx, 'mean_test_score']

print("\nBest Random Forest parameters:", best_params)
print(f"Best Validation Balanced Accuracy: {best_score:.3f}")


Best Random Forest parameters: {'max_depth': 32, 'max_features': 33, 'min_samples_leaf': 1, 'n_estimators': 100, 'random_state': 101}
Best Validation Balanced Accuracy: 0.851


### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [69]:
# Build the best Random Forest using hyperparameters found by grid search
best_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    max_features=best_params['max_features'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=101,
    n_jobs=-1
)
best_forest.fit(X_train, y_train)

# Evaluate
train_bal_acc = balanced_accuracy_score(y_train, best_forest.predict(X_train))
valid_bal_acc = balanced_accuracy_score(y_valid, best_forest.predict(X_valid))
test_bal_acc  = balanced_accuracy_score(y_test,  best_forest.predict(X_test))

print(f"Best Random Forest Performance → Train {train_bal_acc:.3f} | Val {valid_bal_acc:.3f} | Test {test_bal_acc:.3f}")


Best Random Forest Performance → Train 0.964 | Val 0.851 | Test 0.837


What is the value of max_features of your best forest?
 - A: The best Random Forest used max_features = 33. This means each tree considered 33 random features at every split when selecting the best split.

What is the maximum possible value for max_features for this dataset? Why is it beneficial to tune this hyperparameter?
- A: The dataset has 7729 features, so the maximum possible value for max_features is 7729.
- Tuning max_features is important because it controls the randomness and correlation among trees:
Smaller values --> higher diversity between trees --> better generalization (less overfitting).
Larger values --> trees become more similar --> may overfit.
- Thus, it helps balance the bias–variance trade-off.

When fitting Random Forests, what is the primary trade-off controlled by n_estimators? Can you overfit by setting it too large? Why or why not?
- A: n_estimators controls the number of trees in the forest, trading off computation time vs model stability.
More trees reduce variance and improve stability but increase training time.
You generally cannot overfit by making n_estimators very large. Adding more trees only averages predictions, reducing variance without increasing bias.

### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 3 | 1 | 0.646	|0.645	|0.646|
|best Tree	|32 | 1 | 0.877	|0.732	|0.749|
|simple RandomForest	|3 | 100 | 0.819	|0.797	|0.778|
|best RandomForest	|32 | 100 | 0.964	|0.851	|0.837|

In [70]:

comparison = pd.DataFrame([
    ['simple Tree', 3, 1, 0.646, 0.645, 0.646],  
    ['best Tree', best_params.get('max_depth', '-'), 1, 0.877, 0.732, 0.749],
    ['simple RandomForest', 3, 100, 0.819, 0.797, 0.778],
    ['best RandomForest', best_params['max_depth'], 100, 0.964, 0.851, 0.837],
], columns=['method', 'max depth', 'num trees',
            'train BAcc', 'valid BAcc', 'test BAcc'])

display(comparison)


,method,max depth,num trees,train BAcc,valid BAcc,test BAcc
0,simple Tree,3,1,0.646,0.645,0.646
1,best Tree,32,1,0.877,0.732,0.749
2,simple RandomForest,3,100,0.819,0.797,0.778
3,best RandomForest,32,100,0.964,0.851,0.837


### Conclusions:
1. Random forests generally outperform single decision trees

2. Hyperparameter tuning improves performance for both methods

3. The ensemble approach (RF) provides better generalization

4. Best Random Forest achieves the highest performance on all sets